In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import *

2021.08.08

尝试使用proplot绘制平均图，分为AM和JJA时间段分别绘制平均态

使用pre_process/merge_cmorph_cn051.ipynb 下合并掉cmorph以及cmorph的数据

In [ ]:
dir_in            = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/ordata/"
filename_vr       = "vr_pre_98-17.nc"
filename_rcm      = "rcm_pre_98-17.nc"
filename_obsmerge = "obsmerge_pre_98-17.nc"
# filename_cn051    = "large_CN05.1_Pre_1961_2018_daily_025x025.nc"
filename_cn051    = "sel_CN05.1_Pre_1961_2018_daily_025x025.nc"
filename_cmorph   = 'large_CMORPH_98-17.nc'

ds_or = {}
var = {}
var['vr']     = xr.open_dataset(dir_in + filename_vr)['precip_MPAS']
var['rcm']    = xr.open_dataset(dir_in + filename_rcm  )['precip_MPAS']
var['obsmerge']    = xr.open_dataset(dir_in + filename_obsmerge)['premerge']
var['cmorph']    = xr.open_dataset(dir_in + filename_cmorph)['cmorph'][:,0,:,:]
var['cn05.1']    = xr.open_dataset(dir_in + filename_cn051)['pre']

var['obsmerge'] = var['obsmerge'].reset_coords(names = 'lev', drop = True) # 去除掉obs中多余的lev coords
var['cmorph'] = var['cmorph'].reset_coords(names = 'lev', drop = True) # 去除掉obs中多余的lev coords

# CN05.1 的坐标范围相比其他数据较小，将CN05.1的范围扩展到较大的domain上，并且用NAN填充
empty_da = xr.DataArray(np.empty(shape = var['cmorph'].shape), coords = var['cmorph'].coords)
empty_da[:,:,:] = np.nan
lat_smallcn051 = var['cn05.1'].lat
lon_smallcn051 = var['cn05.1'].lon
empty_da.loc[{"lat" : lat_smallcn051, "lon" : lon_smallcn051}] = var['cn05.1'].values
var['cn05.1'] = empty_da

# change coords

var_list = ['vr', 'rcm', 'obsmerge', 'cmorph', 'cn05.1']
for i in var_list:
    rename_dict = dict(zip(var[i].coords.keys(), var['obsmerge'].coords.keys()))
#     # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var[i] = var[i].rename(rename_dict)
    var[i]._coords = var['obsmerge']._coords
    var[i] = var[i].rename(i)

In [ ]:
var_selmonth = {}
var_selmonth['mjja'] = {}
time_idx_mjja = var['obsmerge'].time.dt.month.isin([5,6,7,8])

for mod_name in ['cn05.1','cmorph','obsmerge', 'vr', 'rcm']:
    var_selmonth['mjja'][mod_name] = var[mod_name].isel(time = time_idx_mjja).mean(dim = 'time')

## 计算指标

### 计算制图表格 2022.02.17

需要分别用 CMORPH CN05.1 CMORPH_OUTSIDE 作为观测计算表格

先去除掉陆地上的CMORPH，构造 cmorph_outside 观测数据集

In [ ]:
import skill_metrics as sm

In [ ]:
# 制作海洋上的CMORPH_OUTSIDE
cmorph_outside_mask = np.isnan(var_selmonth['mjja']['cn05.1'])
var_selmonth['mjja']['cmorph_outside'] = var_selmonth['mjja']['cmorph'].where(cmorph_outside_mask)

In [ ]:
def nested_dict():
    from collections import defaultdict
    return defaultdict(nested_dict)
# 进行mask，并且整理成1D数组，得到1D-pattern，用于后续的计算
# 计算指数的平均值

ind = np.array([0])
pd_templist = []
for observation_set in ['cn05.1','cmorph','cmorph_outside']:
# for observation_set in ['obsmerge']:

    var_1d = {}
    iseason = 'mjja'
    for mod_name in [observation_set, 'vr', 'rcm']:
            # 获取1D的平均态 并以RCM为基础进行NAN的MASK
            index_not_nan = ( ~np.isnan(var_selmonth[iseason]['rcm'])) &\
                (~np.isnan(var_selmonth[iseason][observation_set])) &\
                (~np.isnan(var_selmonth[iseason]['vr']) )
            var_1d_temp = xr.where(index_not_nan, var_selmonth[iseason][mod_name], np.nan).values.ravel()
            var_1d[mod_name] = var_1d_temp[~np.isnan(var_1d_temp)]

    for mod_name in [observation_set, 'vr', 'rcm']:
        temp_mean  = var_1d[mod_name].mean()
        temp_bias  = sm.bias(var_1d[mod_name], var_1d[observation_set])
        temp_rmsd  = sm.rmsd(var_1d[mod_name], var_1d[observation_set])
        temp_ccoef = np.corrcoef(var_1d[mod_name], var_1d[observation_set])[1,0]

        pd_templist.append( pd.DataFrame(data = [['precip',observation_set + "-" + mod_name, temp_mean, temp_bias, temp_ccoef, temp_rmsd]], \
        columns = ['variable','obs-model','mean', 'bias','ccoef','rmsd'], index=  ind) )
        ind = ind + 1

# 合并表格并写出
table_pdconcat = pd.concat(pd_templist)
table_pdconcat.to_csv("./output_table/dailypre_SpatialTaylor_2022.02.17.csv")

### 计算制图表格

弃用；不需要rmsd crmsd；并且需要针对两个观测数据集特别计算

In [ ]:
def nested_dict():
    from collections import defaultdict
    return defaultdict(nested_dict)
# 进行mask，并且整理成1D数组，得到1D-pattern，用于后续的计算
# 计算指数的平均值
var_1d = nested_dict()
for iseason in ['mjja']:
    for mod_name in ['cn05.1','cmorph','obsmerge', 'vr', 'rcm']:
            # 获取1D的平均态 并以RCM为基础进行NAN的MASK
            rcm_not_nan = ( ~np.isnan(var_selmonth['mjja']['rcm'])) &\
                 (~np.isnan(var_selmonth['mjja']['obsmerge'])) &\
                 (~np.isnan(var_selmonth['mjja']['vr']) )
            var_1d_temp = xr.where(rcm_not_nan, var_selmonth[iseason][mod_name], np.nan).values.ravel()
            var_1d[iseason][mod_name] = var_1d_temp[~np.isnan(var_1d_temp)]
            # shape check
            # print(var_1d[iseason][var_name][mod_name].shape)

In [ ]:
import skill_metrics as sm
# 计算泰勒图诸要素 到字典taylor_ts中
# 进行mask，并且整理成1D数组，得到1D-pattern，用于后续的计算
# 计算指数的平均值
taylor_space = nested_dict()
for iseason in ['mjja']:
    for mod_name in ['obsmerge', 'vr', 'rcm']:
        temp_obs = var_1d[iseason]['obsmerge']
        temp_mod = var_1d[iseason][mod_name]
        # taylor count
        taylor_space[iseason][mod_name] = sm.taylor_statistics(temp_mod, temp_obs)

### 合并表格并写出csv文件

In [ ]:
pd_templist = []
ind = np.array([0]) # index的整数
for iseason in ['mjja']:
    for mod_name in ['obsmerge', 'vr', 'rcm']:
        temp_ccoef = taylor_space[iseason][mod_name]['ccoef'][1]
        temp_crmsd = taylor_space[iseason][mod_name]['crmsd'][1]
        temp_sdev  = taylor_space[iseason][mod_name]['sdev'][1]
        temp_mean  = var_1d[iseason][mod_name].mean()
        # temp_bias  = var_1d[iseason][mod_name].mean() - var_1d[iseason]['obsmerge'].mean()
        temp_bias  = sm.bias(var_1d[iseason][mod_name], var_1d[iseason]['obsmerge'])
        temp_rmsd  = sm.rmsd(var_1d[iseason][mod_name], var_1d[iseason]['obsmerge'])
    
        pd_templist.append( pd.DataFrame(data = [['precip',mod_name, temp_mean, temp_bias, temp_ccoef, temp_crmsd, temp_rmsd, temp_sdev]], \
            columns = ['variable','model','mean', 'bias','ccoef','crmsd','rmsd','sdev'], index=  ind) )
        ind = ind + 1

# 合并表格并写出
table_pdconcat = pd.concat(pd_templist)
table_pdconcat.to_csv("./output_table/dailypre_SpatialTaylor_2022.02.14.csv")

In [ ]:
table_pdconcat

## 绘图部分

In [ ]:
# 国内政区图的绘制
# Load the border data, CN-border-La.dat is download from
# https://gmt-china.org/data/CN-border-La.dat
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches

cn_border_file = "/m2data2/yycheng/data_stage/CN-border/CN-border_line/CN-border-La.dat"
with open(cn_border_file) as src:
    context = src.read()
    blocks = [cnt for cnt in context.split('>') if len(cnt) > 0]
    borders = [np.fromstring(block, dtype=float, sep=' ') for block in blocks]


### 画图renew
#### 2021.08.08
cn_border_file + shapefile ，但是存在重叠，尝试消除掉China边界，但是其他邻国边界无法处理；
#### 2021.08.09
不使用cn_border_file ，使用shapefile + coast_line（proplot自带） 的办法
shapefile有一些重叠，不绘制行政区
shapefile重新进行绘制，考虑来自 domain_info 中测试的多个shape file中挑选出地资所（改变了prj方式，具体查看prj后缀文件）进行使用
#### 2021.08.30
添加误差图，并且shapefile中的river挑选长江、黄河绘制

In [ ]:
def border_plot(axs):
    """
    进行行政区划的绘制，通过shapefilereader绘制存档的shp文件，需要传入axs，并逐个绘制
    比较消耗时间，调整完毕后最后添加边界的绘制
    """
    ##---- 直接绘图，从边界文件添加
    # for ax_ind in axs:
    # for line in borders:
    #     axs.plot(line[0::100], line[1::100], lw = 0.5, color='gray',transform=ccrs.Geodetic())
    #     axs.plot(line[0::10], line[1::10], lw = 0.4, color='black',transform=ccrs.Geodetic())
    ##---- 使用shp文件添加
        # shapefile数据下载的位置：
    # http://gaohr.win/site/blogs/2017/2017-04-18-GIS-basic-data-of-China.html
    world_border_shapefile = "/m2data2/yycheng/data_stage/CN-border/World/country.shp"
    river_border_shapefile =  "/raid52/yycheng/MPAS/REFERENCE/MODEL_CONSTANT/R1/" + "hyd1_4l.shp"
    southsea_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "southsea_island.shp"
    ninelines_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
    bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/m2data2/yycheng/data_stage/CN-border/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
        river     = shpreader.Reader(river_border_shapefile, encoding = "gbk")
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # 绘制部分的shapefile
        for region in river.records():
            if (region.attributes['NAME'] in ['黄河','长江']):
                # print("----- draw river! -----")
                # 此处需要使用 [] 让region.geometry可以迭代
                ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)


In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import cmaps
# ----- get filter vars coords-----

lon = var_selmonth['mjja']['vr'].lon.values
lat = var_selmonth['mjja']['vr'].lat.values

#----- create plot -----
fig, axs = plot.subplots(ncols=3 ,nrows=2, proj=('cyl'))
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- 添加海洋以及行政区划 -----
border_plot(axs)

#----- colorbar ticks 统一设置 -----
# cmap = 'gist_ncar'
cmap = cmaps.cmocean_deep
# cmap_diff = cmaps.cmp_flux_r
cmap_diff = cmaps.ncl_default_r
# cmap_diff = cmaps.precip4_diff_19lev

mean_ticks = np.concatenate((np.linspace(1,5,5), np.linspace(6,16,6)), axis=0)
# mean_ticks = np.arange(0,18,2)
# mean_ticks = np.linspace(0,20,21)
print("----- tick levels is : " + str(mean_ticks))
diff_ticks = np.linspace(-4, 15, 20)
# diff_ticks = np.concatenate((np.linspace(-5,0,6), np.linspace(1,10,10), [12,14,16,20]), axis=0)

m_mean = axs[0].contourf(lon, lat, var_selmonth['mjja']['cn05.1'].values,\
levels=mean_ticks,cmap=cmap, extend = 'both', norm = 'segmented')

m_mean = axs[3].contourf(lon, lat, var_selmonth['mjja']['cmorph'].values,\
levels=mean_ticks,cmap=cmap, extend = 'both', norm = 'segmented')

for mod_ind, mod_name in enumerate(['vr','rcm']):
    midnorm = plot.DivergingNorm(vcenter = 0, fair = False)
    m_diff = axs[mod_ind+1].contourf(lon, lat, var_selmonth['mjja'][mod_name].values - var_selmonth['mjja']['cn05.1'].values,\
    levels=diff_ticks,cmap=cmap_diff, norm = midnorm, extend = 'both')
    m_diff = axs[mod_ind+3+1].contourf(lon, lat, var_selmonth['mjja'][mod_name].values - var_selmonth['mjja']['cmorph'].values,\
    levels=diff_ticks,cmap=cmap_diff, norm = midnorm, extend = 'both')


#----- add color bar-----
# fig.colorbar(m_overlay, loc='b', cmap=cmap, width=0.1)

fig.colorbar(m_mean, loc='r', width=0.1,
ticklabelsize=5,ticks=mean_ticks, title='precipitation [mm/day]', extend = 'both')
fig.colorbar(m_diff, loc='r', width=0.1,
ticklabelsize=5,ticks=diff_ticks, title='bias' + '[mm/day]', extend = 'both')


# ----- format setting -----
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = True,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
# cartopyautoextent = True, 
# borderslinewidth=.5,
labels = True,
longrid  = True,
latgrid  = True,
#-----GEO axis-----
lonlim=(70, 140), latlim=(5, 60),
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,142,10),
latlocator = np.arange(5,70+2,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(5,70+2,2),
#-----line label-----
# linewidth = 0.5,
suptitle="precipitation",
)

# ----- add axses title and labels setting -----
axs.format(titleweight = 'bold')
axs[0].format(title = "CN05.1")
axs[1].format(title = "VR")
axs[2].format(title = "RCM")

axs[3].format(title = "CMORPH")
axs[4].format(title = "VR")
axs[5].format(title = "RCM")

#----- save figure -----
fig.patch.set_facecolor('white')
plt.savefig('./output_pic/pre_meanstates.2022.02.13.png', dpi=600, facecolor= "white")